In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from scipy.io import loadmat
from scipy.stats import norm
import pickle

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import hsv_to_rgb
from matplotlib.patches import Circle

In [ ]:
from bayesee.evaluation import *

In [ ]:
%load_ext autoreload
%autoreload 2
plt.style.use('bayesee.academic')

In [ ]:
repo_path = Path.cwd().parents[0]
print(repo_path)

In [ ]:
subject = "AZ"
file_name = repo_path / f"data/covert-search/large-field/p2_data_{subject}.pickle"

with open(file_name, "rb") as f:
    stimulus, response = pickle.load(f)

metadata = stimulus["metadata"]
spot_centers = metadata["spot_centers"]
monitor_width, monitor_height = metadata["monitor_size"]
stimulus_size = metadata["stimulus_size"]
n_location = metadata["n_location"]
spot_size = metadata["spot_size"]
stimulus_ppd = metadata["stimulus_ppd"]

target_location = stimulus["df"]["location"]
response_location = response["df"]["response_location"]

accurate_response = target_location == response_location

In [ ]:
shifted_spot_center = metadata["spot_centers"].copy()
shifted_spot_center[:, 0] += (monitor_height - stimulus_size) // 2
shifted_spot_center[:, 1] += (monitor_width - stimulus_size) // 2

list_spot_region = [np.zeros(monitor_width, monitor_height)] * (n_location - 1)
pixel_col, pixel_row = np.meshgrid(np.arange(monitor_width), np.arange(monitor_height))

for location_index in range(n_location - 1):
    list_spot_region[location_index] = (
        pixel_row - shifted_spot_center[location_index, 0]
    ) ** 2 + (
        pixel_col - shifted_spot_center[location_index, 1]
    ) ** 2 <= spot_size**2 / 4

In [ ]:
stimulus_region = np.zeros((monitor_height, monitor_width))

for location_index in range(n_location - 1):
    stimulus_region[list_spot_region[location_index]] = 1

fig, ax = plt.subplots(figsize=(6, 4))
ax.imshow(stimulus_region)

for location_index in range(n_location - 1):
    ax.text(
        shifted_spot_center[location_index, 1],
        shifted_spot_center[location_index, 0],
        f"{location_index+1}",
        ha="center",
        va="center",
    )

In [ ]:
for session_index in stimulus["df"]["session"].unique():
    session_accuracy = accurate_response[
        stimulus["df"]["session"] == session_index
    ].mean()
    print(f"Session {session_index} accuracy: {session_accuracy:.4f}")

print(f"Overall accuracy: {accurate_response.mean():.4f}")

In [ ]:
array_eccentral_distance = np.zeros((n_location - 1,))

for location_index in range(n_location - 1):
    array_eccentral_distance[location_index] = np.sqrt(
        (spot_centers[location_index, 0] - stimulus_size // 2) ** 2
        + (spot_centers[location_index, 1] - stimulus_size // 2) ** 2
    )

array_eccentral_distance /= stimulus_ppd

pixel_precision_array_eccentral_distance = array_eccentral_distance.copy()

for index_d1, distance1 in enumerate(np.unique(array_eccentral_distance)):
    for index_d2, distance2 in enumerate(
        np.unique(array_eccentral_distance)[index_d1 + 1 :]
    ):
        if distance1 != distance2 and np.abs(distance2 - distance1) < 0.5:
            pixel_precision_array_eccentral_distance[
                array_eccentral_distance == distance2
            ] = distance1

In [ ]:
array_orientation = np.ones((n_location - 1,)) * -1

for location_index in range(1, n_location - 1):
    dy = spot_centers[location_index, 0] - stimulus_size // 2
    dx = stimulus_size // 2 - spot_centers[location_index, 1]
    array_orientation[location_index] = (1 - np.arctan2(dy, dx) / np.pi) / 2

In [ ]:
spatial_statistics = pd.DataFrame({"location": range(1, n_location)})
spatial_statistics["ecc"] = pixel_precision_array_eccentral_distance.round(3)
spatial_statistics["orientation"] = array_orientation.round(3)

In [ ]:
for session_index in stimulus["df"]["session"].unique():
    session_condition = stimulus["df"]["session"] == session_index
    session_target_location = stimulus["df"].loc[session_condition, "location"]
    session_response_location = response["df"].loc[
        session_condition, "response_location"
    ]

    hit_rate = np.array(
        [
            sum(
                (session_target_location == location_index)
                & (session_response_location == location_index)
            )
            / sum(session_target_location == location_index)
            for location_index in range(1, n_location)
        ]
    )

    cr_rate = np.array(
        [
            sum(
                (stimulus["df"]["detection_focus"] == location_index)
                & (session_target_location == 0)
                & (session_response_location == 0)
            )
            / sum(
                (stimulus["df"]["detection_focus"] == location_index)
                & (session_target_location == 0)
            )
            for location_index in range(1, n_location)
        ]
    )

    spatial_statistics[f"hit_rate_session{session_index}"] = hit_rate
    spatial_statistics[f"cr_rate_session{session_index}"] = cr_rate

hit_rate = np.array(
    [
        sum((target_location == location_index) & (response_location == location_index))
        / sum(target_location == location_index)
        for location_index in range(1, n_location)
    ]
)

cr_rate = np.array(
    [
        sum(
            (stimulus["df"]["detection_focus"] == location_index)
            & (target_location == 0)
            & (response_location == 0)
        )
        / sum(
            (stimulus["df"]["detection_focus"] == location_index)
            & (target_location == 0)
        )
        for location_index in range(1, n_location)
    ]
)

spatial_statistics["hit_rate"] = hit_rate
spatial_statistics["cr_rate"] = cr_rate

n_present = np.array(
    [sum(target_location == location_index) for location_index in range(1, n_location)]
)

n_absent = np.array(
    [
        sum(
            (stimulus["df"]["detection_focus"] == location_index)
            & (target_location == 0)
        )
        for location_index in range(1, n_location)
    ]
)

spatial_statistics["n_present"] = n_present
spatial_statistics["n_absent"] = n_absent

In [ ]:
hit_rate_for_dp = hit_rate.copy()
cr_rate_for_dp = cr_rate.copy()

## Assumption: if we were to double the number of trials, one error would be made
tol = 1e-5
for location_index in range(1, n_location):
    if abs(hit_rate[location_index - 1] - 1.0) < tol:
        hit_rate_for_dp[location_index - 1] = 1 - 1 / (
            n_present[location_index - 1] * 2
        )

    if abs(cr_rate[location_index - 1] - 1.0) < tol:
        cr_rate_for_dp[location_index - 1] = 1 - 1 / (n_absent[location_index - 1] * 2)

dp = norm.ppf(hit_rate_for_dp) - norm.ppf(1 - cr_rate_for_dp)
pc_max = compute_PC_max_w_dprime(dp)
spatial_statistics["dp"] = dp.round(4)
spatial_statistics["pc_max"] = pc_max.round(4)

In [ ]:
spatial_statistics.to_csv(
    repo_path
    / f"data/covert-search/large-field/derived/p2_spatial_statistics_{subject}.csv",
    index=False,
)

In [ ]:
orientation_hsv_color_map = np.zeros((n_location - 1, 3))
orientation_hsv_color_map[:, 0] = array_orientation

for location_index in range(n_location - 1):
    orientation_hsv_color_map[location_index, 1] = 1

    if location_index == 0:
        orientation_hsv_color_map[location_index, 2] = 0
    else:
        orientation_hsv_color_map[location_index, 2] = 0.75

rgb_orientation_color_map = hsv_to_rgb(orientation_hsv_color_map)

orientation_color_palette = dict(
    zip(spatial_statistics["orientation"], rgb_orientation_color_map)
)

In [ ]:
interested_statistics = (
    ["hit_rate", "cr_rate", "dp", "pc_max"]
    + [
        f"hit_rate_session{session_index}"
        for session_index in stimulus["df"]["session"].unique()
    ]
    + [
        f"cr_rate_session{session_index}"
        for session_index in stimulus["df"]["session"].unique()
    ]
)

errors_interested_statistics = [None for _ in range(len(interested_statistics))]
errors_interested_statistics[0] = 2 * np.sqrt(hit_rate * (1 - hit_rate) / n_present)
errors_interested_statistics[1] = 2 * np.sqrt(cr_rate * (1 - cr_rate) / n_absent)
errors_interested_statistics[3] = 2 * np.sqrt(
    pc_max * (1 - pc_max) / (n_present + n_absent)
)

for statistic_index, statistic in enumerate(interested_statistics):
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 6))

    bar_width = 0.1

    df_sorted = spatial_statistics.sort_values(by=["ecc", "orientation"], kind="stable")
    for ecc_index, ecc in enumerate(df_sorted["ecc"].unique()):
        ecc_condition = df_sorted["ecc"] == ecc
        orientation_sorted = df_sorted.loc[ecc_condition, "orientation"]
        n_x_bar = len(orientation_sorted)
        x_bar = np.linspace(
            ecc_index - (n_x_bar - 1) * bar_width / 2,
            ecc_index + (n_x_bar + 1) * bar_width / 2,
            n_x_bar,
        )
        y_bar = df_sorted.loc[ecc_condition, statistic]

        if errors_interested_statistics[statistic_index] is None:
            y_error = None
        else:
            y_error = errors_interested_statistics[statistic_index][df_sorted.index][
                ecc_condition
            ]

        ax.bar(
            x_bar,
            y_bar,
            yerr=y_error,
            color=[
                orientation_color_palette[orientation]
                for orientation in np.sort(orientation_sorted, kind="stable")
            ],
            width=bar_width,
            error_kw={"elinewidth": 3, "capsize": 6, "capthick": 3, "alpha": 0.75},
        )

    ax.set(
        xlabel="Eccentral distance (deg)",
        ylabel=statistic,
        xticks=range(len(df_sorted["ecc"].unique())),
    )

    ax.set_xticklabels(df_sorted["ecc"].unique())

    inner_ax = fig.add_axes([-0.025, -0.1, 0.144, 0.2405])
    inner_ax.imshow(stimulus_region)
    inner_ax.set(xticks=[], yticks=[])

    for location_index in range(n_location - 1):
        circle = Circle(
            (
                shifted_spot_center[location_index, 1],
                shifted_spot_center[location_index, 0],
            ),
            spot_size / 2,
            color=rgb_orientation_color_map[location_index],
            linewidth=1,
        )
        inner_ax.add_patch(circle)

    plt.show()